# Data Visualization HW1 (Python)
(student: ***Albert Hakobyan***)

In [234]:
import pandas as pd
import numpy as np

data = pd.read_csv('crime_data.csv')
 
missing_counts = data.isna().sum()
print(missing_counts)

missing_cols = missing_counts[missing_counts > 0]
print(missing_cols)

cols_to_drop = missing_counts[missing_counts > 0.5 * len(data)].index.tolist() # Conversion from series to list was necessary here

# The following loop checks if more than 50% of LAT/LON values are 0, add them to drop list 
# (These null values were described to be treated as missing values in the HW document)

for column in ['LAT', 'LON']:
    if (data[column] == 0).sum() > 0.5 * len(data):
        cols_to_drop.append(col)

data_filtered = data.drop(columns = cols_to_drop)

print("Dropped columns:", cols_to_drop)
print(data_filtered.head())

DR_NO                 0
Date Rptd             0
DATE OCC              0
TIME OCC              0
AREA                  0
AREA NAME             0
Rpt Dist No           0
Part 1-2              0
Crm Cd                0
Crm Cd Desc           0
Mocodes            7498
Vict Age              0
Vict Sex           7163
Vict Descent       7165
Premis Cd             0
Premis Desc          29
Weapon Used Cd    33654
Weapon Desc       33654
Status                0
Status Desc           0
Crm Cd 1              2
Crm Cd 2          46448
Crm Cd 3          49885
Crm Cd 4          49995
LOCATION              0
Cross Street      42258
LAT                   0
LON                   0
dtype: int64
Mocodes            7498
Vict Sex           7163
Vict Descent       7165
Premis Desc          29
Weapon Used Cd    33654
Weapon Desc       33654
Crm Cd 1              2
Crm Cd 2          46448
Crm Cd 3          49885
Crm Cd 4          49995
Cross Street      42258
dtype: int64
Dropped columns: ['Weapon Used Cd', 'W

In [7]:
data_filtered['DATE OCC'] = pd.to_datetime(data_filtered['DATE OCC'], format='%m/%d/%Y %H:%M:%S %p')
data_filtered['YEAR'] = data_filtered['DATE OCC'].dt.year
data_filtered['MONTH'] = data_filtered['DATE OCC'].dt.month
data_filtered['DAY'] = data_filtered['DATE OCC'].dt.day

data_filtered['TIME OCC'] = data_filtered['TIME OCC'].astype(str)
data_filtered['HOUR'] = data_filtered['TIME OCC'].str[:2].astype(int)

print(data_filtered.head())
print(data_filtered.dtypes)

       DR_NO               Date Rptd            DATE OCC TIME OCC  AREA  \
0  241711715  08/01/2024 12:00:00 AM 2024-08-01 12:00:00     1319    17   
1  231014031  09/21/2023 12:00:00 AM 2023-09-15 12:00:00     1930    10   
2  231010808  06/27/2023 12:00:00 AM 2023-06-26 12:00:00     1230    10   
3  211410441  04/25/2021 12:00:00 AM 2021-04-25 12:00:00     2330    14   
4  211114569  10/25/2021 12:00:00 AM 2021-10-25 12:00:00     1455    11   

     AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0   Devonshire         1791         1     440   
1  West Valley         1011         2     354   
2  West Valley         1015         2     354   
3      Pacific         1488         2     626   
4    Northeast         1123         1     210   

                          Crm Cd Desc  ... Status  Status Desc Crm Cd 1  \
0  THEFT PLAIN - PETTY ($950 & UNDER)  ...     IC  Invest Cont    440.0   
1                   THEFT OF IDENTITY  ...     IC  Invest Cont    354.0   
2                   THEFT OF 

In [238]:
filtered_data = data_filtered[(data_filtered['YEAR'] == 2023) & (data_filtered['Crm Cd Desc'] == 'BURGLARY')]

print(filtered_data.head())

         DR_NO               Date Rptd            DATE OCC TIME OCC  AREA  \
47   231107877  04/15/2023 12:00:00 AM 2023-01-15 12:00:00     0500    11   
147  231912840  08/15/2023 12:00:00 AM 2023-08-14 12:00:00     2200    19   
292  230813484  08/19/2023 12:00:00 AM 2023-08-19 12:00:00     0510     8   
317  230126836  12/20/2023 12:00:00 AM 2023-12-10 12:00:00     1200     1   
326  231506351  02/26/2023 12:00:00 AM 2023-02-22 12:00:00     1230    15   

       AREA NAME  Rpt Dist No  Part 1-2  Crm Cd Crm Cd Desc  ... Status  \
47     Northeast         1151         1     310    BURGLARY  ...     IC   
147      Mission         1962         1     310    BURGLARY  ...     IC   
292      West LA          857         1     310    BURGLARY  ...     IC   
317      Central          154         1     310    BURGLARY  ...     IC   
326  N Hollywood         1562         1     310    BURGLARY  ...     IC   

     Status Desc Crm Cd 1                                  LOCATION      LAT  \
47   I

In [240]:
crime_stats = filtered_data.groupby('AREA NAME').agg(total_crimes=('DR_NO', 'count'),
    avg_vict_age=('Vict Age', lambda x: np.floor(x.dropna().mean())))

crime_stats_grouped = crime_stats.sort_values(by='total_crimes', ascending=False)

print(crime_stats_grouped.head(10))

             total_crimes  avg_vict_age
AREA NAME                              
Devonshire             58          43.0
West LA                58          40.0
Olympic                53          30.0
West Valley            52          35.0
N Hollywood            50          29.0
Wilshire               48          39.0
Pacific                48          24.0
Van Nuys               44          41.0
Southwest              32          35.0
77th Street            31          30.0


In [228]:
top3_crimes = data_filtered['Crm Cd Desc'].value_counts().nlargest(3)
print("The top 3 crimes are: \n", top3_crimes)

crimes_by_hour = data_filtered.groupby('HOUR').agg(total_crimes = ('DR_NO', 'count'))
print(crimes_by_hour)

crimes_by_sex = data_filtered.groupby('Vict Sex').agg(total_crimes = ('DR_NO', 'count'),
    avg_victim_age = ('Vict Age', lambda x: np.floor(x.dropna().mean())))

print(crimes_by_sex)

The top 3 crimes are: 
 Crm Cd Desc
VEHICLE - STOLEN            5733
BATTERY - SIMPLE ASSAULT    3715
THEFT OF IDENTITY           3169
Name: count, dtype: int64
      total_crimes
HOUR              
0             2025
1             1490
2             1241
3             1063
4              975
5              848
6             1106
7             1302
8             1776
9             1841
10            2110
11            2114
12            3446
13            2246
14            2504
15            2667
16            2597
17            2914
18            3015
19            2762
20            2830
21            2553
22            2469
23            2106
          total_crimes  avg_victim_age
Vict Sex                              
F                17922            38.0
H                    3            36.0
M                20076            37.0
X                 4836             2.0


In [242]:
# This condition drops duplicate 'Weapon Used Cd' if it already exists (for code reproducability purposes/hw checking)
if 'Weapon Used Cd' in filtered_data.columns:
    filtered_data = filtered_data.drop(columns=['Weapon Used Cd'])

filtered_data = filtered_data.merge(
    data[['DR_NO', 'Weapon Used Cd']], 
    on='DR_NO', 
    how='left'
)

filtered_data['Severity_Score'] = np.where(
    filtered_data['Weapon Used Cd'].notna(), 5,
    np.where(filtered_data['Crm Cd Desc'] == 'BURGLARY', 3, 1)  # If BURGLARY → 3, otherwise 1
)

severity_score_by_area = filtered_data.groupby('AREA NAME').agg(
    total_severity_score=('Severity_Score', 'sum')
).reset_index()


print(severity_score_by_area.head())

     AREA NAME  total_severity_score
0  77th Street                   103
1      Central                    95
2   Devonshire                   212
3     Foothill                    58
4       Harbor                    66


In [244]:
print(filtered_data.head(5))

       DR_NO               Date Rptd            DATE OCC TIME OCC  AREA  \
0  231107877  04/15/2023 12:00:00 AM 2023-01-15 12:00:00     0500    11   
1  231912840  08/15/2023 12:00:00 AM 2023-08-14 12:00:00     2200    19   
2  230813484  08/19/2023 12:00:00 AM 2023-08-19 12:00:00     0510     8   
3  230126836  12/20/2023 12:00:00 AM 2023-12-10 12:00:00     1200     1   
4  231506351  02/26/2023 12:00:00 AM 2023-02-22 12:00:00     1230    15   

     AREA NAME  Rpt Dist No  Part 1-2  Crm Cd Crm Cd Desc  ... Crm Cd 1  \
0    Northeast         1151         1     310    BURGLARY  ...    310.0   
1      Mission         1962         1     310    BURGLARY  ...    310.0   
2      West LA          857         1     310    BURGLARY  ...    310.0   
3      Central          154         1     310    BURGLARY  ...    310.0   
4  N Hollywood         1562         1     310    BURGLARY  ...    310.0   

                                   LOCATION      LAT       LON  YEAR MONTH  \
0   5000 W  SUNSET  

In [246]:
print(filtered_data.tail(5))

         DR_NO               Date Rptd            DATE OCC TIME OCC  AREA  \
720  231606276  03/14/2023 12:00:00 AM 2023-03-14 12:00:00     0001    16   
721  230811878  07/13/2023 12:00:00 AM 2023-06-23 12:00:00     1200     8   
722  231320692  11/16/2023 12:00:00 AM 2023-11-11 12:00:00     1600    13   
723  231515412  09/28/2023 12:00:00 AM 2023-09-26 12:00:00     1530    15   
724  230500901  11/12/2023 12:00:00 AM 2023-11-12 12:00:00     1315     5   

       AREA NAME  Rpt Dist No  Part 1-2  Crm Cd Crm Cd Desc  ... Crm Cd 1  \
720     Foothill         1677         1     310    BURGLARY  ...    310.0   
721      West LA          807         1     310    BURGLARY  ...    310.0   
722       Newton         1303         1     310    BURGLARY  ...    310.0   
723  N Hollywood         1547         1     310    BURGLARY  ...    310.0   
724       Harbor          515         1     310    BURGLARY  ...    310.0   

                                     LOCATION      LAT       LON  YEAR MON